# Lecture Plan

1. Chains
   - Simple Sequenctial Chain
   - Sequential Chain
   - Router Chain
   - Math Chain
   - Tranform Chain
2. Document Loaders
3. Transformers
4. Text Embeddings
5. Vector Store
6. Retreivers
7. Multi Query Retreival


---

### LangChain Expression Language (LCEL)


- Easy way to compose chains together using Runnable Interface
  
Provides:

- Streaming
- Async
- Parallel Execution
- Retires and Fallbacks
- Input and Output Schemas
- Integration with langsmith

#### Runnable Interface

- strem
- invoke
- batch

Async:

- astrem
- ainvoke
- abatch


[LCEL Reference](https://python.langchain.com/docs/expression_language/interface)

## Runnable Interface


1. Runnable Parallel: used to define  and run multiple values and operations in parallel.

2. Runnble Passthrough: used to take the input and pass it through 

3. Runnable Lambda: used to turn the python functions to pipe compatable functions

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

#### LLM Chain

In [2]:
from langchain.chat_models.openai import ChatOpenAI

model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)

c:\TrainingMaterial\generative-ai\genai-material\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
from langchain.llms.deepinfra import DeepInfra
from langchain_experimental.chat_models import Llama2Chat
llm = DeepInfra(
    model_id="meta-llama/Llama-2-70b-chat-hf"
)

model = Llama2Chat(llm=llm)

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate


prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a code assistant that generates code for users input"),
        HumanMessagePromptTemplate.from_template("provide code for {text} in {language}")
    ]
)

In [11]:
# LLM Chain

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = prompt | model | parser

chain.invoke({'text':'print prime numbers', 'language':'python'})

'Sure! Here\'s a code snippet that prints all prime numbers within a given range in Python:\n\n```python\ndef is_prime(num):\n    if num < 2:\n        return False\n    for i in range(2, int(num ** 0.5) + 1):\n        if num % i == 0:\n            return False\n    return True\n\nstart = int(input("Enter the starting number: "))\nend = int(input("Enter the ending number: "))\n\nprint("Prime numbers between", start, "and", end, "are:")\nfor num in range(start, end + 1):\n    if is_prime(num):\n        print(num)\n```\n\nIn this code, we define a helper function `is_prime()` that checks whether a number is prime or not. Then, we take user input for the starting and ending numbers of the range. Finally, we iterate through the range and print all the prime numbers within that range.'

### Simple Sequential Chain

![Simple Sequential Chain](./images/SimpleSequentialChain.PNG)

In [41]:
company_name_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are an assistant that provides a single company name based on the description provided by the user"),
        HumanMessagePromptTemplate.from_template("{text}")

    ]
)

slogan_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="genrate a slogan from company name provided by the user and print it along with the name of the company"),
        HumanMessagePromptTemplate.from_template("{company_name}")

    ]
)

In [42]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    company_name = company_name_prompt | model,
    company_slogan = {'company_name' : company_name_prompt | model} | slogan_prompt_template | model | parser 
)
# chain = {'company_name' : company_name_prompt | model} | slogan_prompt_template | model | parser 


In [43]:
chain.invoke({'text':'a company that sells flying cars'})

{'company_name': AIMessage(content='  Sure! Based on your description, the company name that comes to mind is "Terrafugia". Terrafugia is a company that designs and manufactures flying cars, also known as personal aerial vehicles or PAVs. The company was founded in 2006 and is based in Woburn, Massachusetts, USA. Terrafugia\'s mission is to create practical and safe flying cars that can be used for personal transportation and recreation. Their first product, the Transition, is a light aircraft that can convert from a car to a plane in just a few minutes.'),
 'company_slogan': '  Sure, here\'s a slogan for Terrafugia:\n\n"Terrafugia: Take to the skies, and hit the road"\n\nOr, if you prefer, here\'s another option:\n\n"Terrafugia: The sky\'s the limit, and so is the highway"\n\nI hope these options capture the essence of Terrafugia\'s innovative technology and the freedom it offers to its customers!'}

#### Sequential Chain

![Sequential Chain](./images/SequentialChain.PNG)

In [4]:
code_generation_prompt = ChatPromptTemplate.from_messages(
   [
       SystemMessage(content="You are a code assistant that generates code for users input"),
       HumanMessagePromptTemplate.from_template("generate code for {text} in {language}")

   ] 
)

time_complexity_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a code assistant that evaluates the time and space complexity of code for users input"),
        HumanMessagePromptTemplate.from_template("{code}")
    ]
)


code_summary_chain = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="you are a code assitant that prints the time complexity of the code and code provided by the user and an explanation for the code"),
        HumanMessagePromptTemplate.from_template("{code} and {time_complexity} ")
    ]
)


In [10]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain = RunnableParallel(
    code_generation = code_generation_prompt | model,
    code_complexity = RunnablePassthrough.assign(code = (code_generation_prompt | model))| time_complexity_prompt | model,
    code_comparision = {'code' : (code_generation_prompt | model), 'time_complexity': ({'code' : code_generation_prompt | model} | time_complexity_prompt | model)} | code_summary_chain | model
)

In [11]:
chain.invoke({'text':'print prime numbers', 'language':'python'})

{'code_generation': AIMessage(content="  Sure! Here's an example code that prints all prime numbers up to a given number `n`:\n```\ndef print_primes(n):\n    for i in range(2, n+1):\n        is_prime = True\n        for j in range(2, int(i**0.5) + 1):\n            if i % j == 0:\n                is_prime = False\n                break\n        if is_prime:\n            print(i)\n```\nHere's an explanation of the code:\n\n1. The function `print_primes` takes an integer `n` as input.\n2. The loop `for i in range(2, n+1)` iterates over the numbers from 2 to `n`.\n3. The variable `is_prime` is initialized to `True` for each iteration.\n4. The loop `for j in range(2, int(i**0.5) + 1)` checks whether `i` is divisible by any number between 2 and the square root of `i`. If `i` is divisible by `j`, then `is_prime` is set to `False` and the loop breaks.\n5. If `is_prime` is still `True` after the inner loop, then `i` is a prime number and it is printed.\n\nHere's an example usage of the function

#### Router Chain

![Router Chain](./images/RouterChain.PNG)

In [22]:
sales_prompt = ChatPromptTemplate.from_messages(

    [

    SystemMessage(content="You are a helpfull, honest sales assisnt for a telecom company called brightspeed"),
    HumanMessagePromptTemplate.from_template("{text}")

    ]
    
)

service_prompt = ChatPromptTemplate.from_messages(

    [
    SystemMessage(content="You are a helpfull, honest service assisnt for a telecom company called brightspeed guide the user to resolve the issues."),
    HumanMessagePromptTemplate.from_template("{text}")
    ]
)



router_prompt = ChatPromptTemplate.from_messages(

    [
    SystemMessage(content="Categorize the user input to sales or service along with actual text"),
    HumanMessagePromptTemplate.from_template("{text}")
    ]
)

In [23]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

from transformers import pipeline


# classifier = pipeline('zero-shot-classification', candidate_labels = ['sales', 'service'])

parser = StrOutputParser()

chain = router_prompt | model | parser

def categorize_text(text):
    if("sales" in text):
        return sales_prompt
    else:
        return service_prompt
    

router_chain = RunnablePassthrough.assign(text = (router_prompt | model | parser)) | RunnableLambda(lambda text: categorize_text(text)) | model

In [24]:
router_chain.invoke({'text':"What are the broadband plans available?"})

AIMessage(content="  Hello! Welcome to Brightspeed! We're happy to help you with your inquiry.\n\nWe offer a variety of broadband plans that cater to different needs and budgets. Our plans come with different speeds, data caps, and features to ensure you get the best experience for your online activities.\n\nHere are some of our most popular broadband plans:\n\n1. Basic: This plan offers speeds of up to 50 Mbps and a data cap of 500 GB. It's perfect for light internet users who primarily use the internet for browsing, emailing, and social media.\n2. Standard: With speeds of up to 100 Mbps and a data cap of 1 TB, this plan is ideal for households with multiple devices. It's great for streaming, online gaming, and heavy internet usage.\n3. Premium: Our top-tier plan offers speeds of up to 500 Mbps and a data cap of 2 TB. It's designed for heavy internet users who need fast speeds for streaming, online gaming, and large file transfers.\n\nWe also offer customized plans for businesses, so 

#### Transform Chain

#### Math Chain